<a href="https://colab.research.google.com/github/iqbal899/DBMSassignment/blob/main/DBMS_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install transformers torch
!pip install accelerate
!pip install sentencepiece # Some models might require this

In [28]:
from google.colab import userdata
import os
# Set the environment variable for Hugging Face
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [29]:
db_schema = """
CREATE TABLE student (
    student_id INTEGER,
    name VARCHAR,
    marks INTEGER,
    grade VARCHAR,
    age INTEGER
);
"""
from transformers import pipeline
model_name = "gaussalgo/T5-LM-Large-text2sql-spider"
text_to_sql_pipeline = pipeline("text2text-generation", model=model_name)


Device set to use cpu


In [30]:
# Install pandasql
!pip install pandasql

import pandas as pd
import numpy as np
from pandasql import sqldf

# Set a seed for reproducibility
np.random.seed(42)

# Create a DataFrame named 'student' to match the schema
student = pd.DataFrame({
    'student_id': np.arange(1, 21),
    'name': [f'Student_{i}' for i in range(1, 21)],
    'marks': np.random.randint(50, 100, 20),
    'grade': np.random.choice(['A', 'B', 'C', 'D'], 20),
    'age': np.random.randint(18, 25, 20)
})

# Display the first few rows
print("Dummy Student DataFrame:")
print(student.head())

# Define a function to execute SQL on the DataFrame
def psql(q):
    return sqldf(q, globals())

Dummy Student DataFrame:
   student_id       name  marks grade  age
0           1  Student_1     88     B   21
1           2  Student_2     78     B   24
2           3  Student_3     64     B   23
3           4  Student_4     92     D   23
4           5  Student_5     57     D   24


In [31]:
# Set Pandas options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Print the entire student DataFrame
print(student)

    student_id        name  marks grade  age
0            1   Student_1     88     B   21
1            2   Student_2     78     B   24
2            3   Student_3     64     B   23
3            4   Student_4     92     D   23
4            5   Student_5     57     D   24
5            6   Student_6     70     A   23
6            7   Student_7     88     A   20
7            8   Student_8     68     D   21
8            9   Student_9     72     B   24
9           10  Student_10     60     B   21
10          11  Student_11     60     A   18
11          12  Student_12     73     D   20
12          13  Student_13     85     A   22
13          14  Student_14     89     A   20
14          15  Student_15     73     C   24
15          16  Student_16     52     C   22
16          17  Student_17     71     C   18
17          18  Student_18     51     B   24
18          19  Student_19     73     D   19
19          20  Student_20     93     D   21


**V1_QUERY**

In [32]:
nl_query_v1 = "Show all students with marks above 80."
prompt_v1 = f"### {db_schema}\n### Task\n{nl_query_v1}"
sql_v1 = text_to_sql_pipeline(prompt_v1)

print(f"V1 Natural Language: {nl_query_v1}")
print(f"V1 Generated SQL: {sql_v1[0]['generated_text']}")

# Execute the generated SQL on the dummy data
result_v1 = psql(sql_v1[0]['generated_text'])
print("V1 Query Result on Dummy Data:")
print(result_v1)

V1 Natural Language: Show all students with marks above 80.
V1 Generated SQL: SELECT name FROM student WHERE marks > 80
V1 Query Result on Dummy Data:
         name
0   Student_1
1   Student_4
2   Student_7
3  Student_13
4  Student_14
5  Student_20


#***realted queries for table student***

In [33]:
nl_query = "Count the number of students for each grade."
prompt = f"### {db_schema}\n### Task\n{nl_query}"
sql = text_to_sql_pipeline(prompt)
print(f"Generated SQL: {sql[0]['generated_text']}")
result = psql(sql[0]['generated_text'])
print("Result:")
print(result)

Generated SQL: SELECT count(*) , grade FROM student GROUP BY grade
Result:
   count(*) grade
0         5     A
1         6     B
2         3     C
3         6     D


In [34]:
#This query uses a simple COUNT function to find the total number of students.
nl_query_count = "How many students are there?"
prompt_count = f"### {db_schema}\n### Task\n{nl_query_count}"
sql_count = text_to_sql_pipeline(prompt_count)

print(f"Count Query: {nl_query_count}")
print(f"Generated SQL: {sql_count[0]['generated_text']}")

# Execute the generated SQL
result_count = psql(sql_count[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result_count)

Count Query: How many students are there?
Generated SQL: SELECT count(*) FROM student
Query Result on Dummy Data:
   count(*)
0        20


In [35]:
#TO GET ALL DATA
# Updated Natural Language Query for clarity
nl_query_all = "Show all data from the student table."

# The prompt now explicitly tells the model to use the "student" table
prompt_all = f"### {db_schema}\n### Task\n{nl_query_all}"
sql_all = text_to_sql_pipeline(prompt_all)

print(f"All Records Query: {nl_query_all}")
print(f"Generated SQL: {sql_all[0]['generated_text']}")

# Execute the generated SQL
# This will now work correctly as the SQL should not contain a JOIN
result_all = psql(sql_all[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result_all)

All Records Query: Show all data from the student table.
Generated SQL: SELECT * FROM student
Query Result on Dummy Data:
    student_id        name  marks grade  age
0            1   Student_1     88     B   21
1            2   Student_2     78     B   24
2            3   Student_3     64     B   23
3            4   Student_4     92     D   23
4            5   Student_5     57     D   24
5            6   Student_6     70     A   23
6            7   Student_7     88     A   20
7            8   Student_8     68     D   21
8            9   Student_9     72     B   24
9           10  Student_10     60     B   21
10          11  Student_11     60     A   18
11          12  Student_12     73     D   20
12          13  Student_13     85     A   22
13          14  Student_14     89     A   20
14          15  Student_15     73     C   24
15          16  Student_16     52     C   22
16          17  Student_17     71     C   18
17          18  Student_18     51     B   24
18          19  Student

In [36]:
#GETTING MINIMUM MARKS

nl_query_min = "What is the lowest mark?"
prompt_min = f"### {db_schema}\n### Task\n{nl_query_min}"
sql_min = text_to_sql_pipeline(prompt_min)
print(f"Min Query: {nl_query_min}")
print(f"Generated SQL: {sql_min[0]['generated_text']}")
result_min = psql(sql_min[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result_min)

Min Query: What is the lowest mark?
Generated SQL: SELECT min(marks) FROM student
Query Result on Dummy Data:
   min(marks)
0          51


In [37]:
nl_query_lowest_student = "Select the name and marks of the student with the lowest mark."
prompt_lowest_student = f"### {db_schema}\n### Task\n{nl_query_lowest_student}"
sql_lowest_student = text_to_sql_pipeline(prompt_lowest_student)

print(f"Lowest Mark Query: {nl_query_lowest_student}")
print(f"Generated SQL: {sql_lowest_student[0]['generated_text']}")

# Execute the generated SQL on the dummy data
result_lowest_student = psql(sql_lowest_student[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result_lowest_student)

Lowest Mark Query: Select the name and marks of the student with the lowest mark.
Generated SQL: SELECT name , marks FROM student ORDER BY marks LIMIT 1
Query Result on Dummy Data:
         name  marks
0  Student_18     51


**QUERY V2**

In [38]:
# The natural language query is changed to be more explicit.
nl_query_v2 = "Show the name and marks and age of students with age greater than 20 and grade 'A'."

# The prompt now explicitly tells the model to use the "student" table without a JOIN.
prompt_v2 = f"### {db_schema}\n### Task\n{nl_query_v2}"
sql_v2 = text_to_sql_pipeline(prompt_v2)

print(f"V2 Natural Language: {nl_query_v2}")
print(f"V2 Generated SQL: {sql_v2[0]['generated_text']}")

# Execute the generated SQL on the dummy data
result_v2 = psql(sql_v2[0]['generated_text'])
print("V2 Query Result on Dummy Data:")
print(result_v2)

V2 Natural Language: Show the name and marks and age of students with age greater than 20 and grade 'A'.
V2 Generated SQL: SELECT name, marks, age FROM student WHERE age > 20 AND grade = 'A'
V2 Query Result on Dummy Data:
         name  marks  age
0   Student_6     70   23
1  Student_13     85   22


**QUERY_V3**

In [39]:
nl_query_v3 = "Find the average marks of students."
prompt_v3 = f"### {db_schema}\n### Task\n{nl_query_v3}"
sql_v3 = text_to_sql_pipeline(prompt_v3)

print(f"V3 Natural Language: {nl_query_v3}")
print(f"V3 Generated SQL: {sql_v3[0]['generated_text']}")

# Execute the generated SQL
result_v3 = psql(sql_v3[0]['generated_text'])
print("V3 Query Result on Dummy Data:")
print(result_v3)

V3 Natural Language: Find the average marks of students.
V3 Generated SQL: SELECT avg(marks) FROM student
V3 Query Result on Dummy Data:
   avg(marks)
0       72.85


#**QUERY_V4**

In [40]:
nl_query_v4 = "Show names ,marks of students  ordered by marks descending."
prompt_v4 = f"### {db_schema}\n### Task\n{nl_query_v4}"
sql_v4 = text_to_sql_pipeline(prompt_v4)

print(f"V4 Natural Language: {nl_query_v4}")
print(f"V4 Generated SQL: {sql_v4[0]['generated_text']}")

# Execute the generated SQL
result_v4 = psql(sql_v4[0]['generated_text'])
print("V4 Query Result on Dummy Data:")
print(result_v4)

V4 Natural Language: Show names ,marks of students  ordered by marks descending.
V4 Generated SQL: SELECT name , marks FROM student ORDER BY marks DESC
V4 Query Result on Dummy Data:
          name  marks
0   Student_20     93
1    Student_4     92
2   Student_14     89
3    Student_1     88
4    Student_7     88
5   Student_13     85
6    Student_2     78
7   Student_12     73
8   Student_15     73
9   Student_19     73
10   Student_9     72
11  Student_17     71
12   Student_6     70
13   Student_8     68
14   Student_3     64
15  Student_10     60
16  Student_11     60
17   Student_5     57
18  Student_16     52
19  Student_18     51


**QUERY_V5**

In [41]:
nl_query_v5 = "Show average marks by grade."
prompt_v5 = f"### {db_schema}\n### Task\n{nl_query_v5}"
sql_v5 = text_to_sql_pipeline(prompt_v5)

print(f"V5 Natural Language: {nl_query_v5}")
print(f"V5 Generated SQL: {sql_v5[0]['generated_text']}")

# Execute the generated SQL
result_v5 = psql(sql_v5[0]['generated_text'])
print("V5 Query Result on Dummy Data:")
print(result_v5)

V5 Natural Language: Show average marks by grade.
V5 Generated SQL: SELECT avg(marks) FROM student GROUP BY grade
V5 Query Result on Dummy Data:
   avg(marks)
0   78.400000
1   68.833333
2   65.333333
3   76.000000


**QUERY_V7**

In [42]:
# V7: Synonym Handling
nl_query_v7 = "List the names of students with marks above 80."
prompt_v7 = f"### {db_schema}\n### Task\n{nl_query_v7}"
sql_v7 = text_to_sql_pipeline(prompt_v7)

print(f"V7 Natural Language: {nl_query_v7}")
print(f"V7 Generated SQL: {sql_v7[0]['generated_text']}")

# Execute the generated SQL
result_v7 = psql(sql_v7[0]['generated_text'])
print("V7 Query Result on Dummy Data:")
print(result_v7)

V7 Natural Language: List the names of students with marks above 80.
V7 Generated SQL: SELECT name FROM student WHERE marks > 80
V7 Query Result on Dummy Data:
         name
0   Student_1
1   Student_4
2   Student_7
3  Student_13
4  Student_14
5  Student_20


**#more_queries**

In [43]:
nl_query = "Show the name , marks of students, ordered by marks in descending order."
prompt = f"### {db_schema}\n### Task\n{nl_query}"
sql = text_to_sql_pipeline(prompt)

print(f"Natural Language: {nl_query}")
print(f"Generated SQL: {sql[0]['generated_text']}")

# Execute the generated SQL
result = psql(sql[0]['generated_text'])
print("Query Result on Dummy Data:")
print(result)

Natural Language: Show the name , marks of students, ordered by marks in descending order.
Generated SQL: SELECT name , marks FROM student ORDER BY marks DESC
Query Result on Dummy Data:
          name  marks
0   Student_20     93
1    Student_4     92
2   Student_14     89
3    Student_1     88
4    Student_7     88
5   Student_13     85
6    Student_2     78
7   Student_12     73
8   Student_15     73
9   Student_19     73
10   Student_9     72
11  Student_17     71
12   Student_6     70
13   Student_8     68
14   Student_3     64
15  Student_10     60
16  Student_11     60
17   Student_5     57
18  Student_16     52
19  Student_18     51


In [44]:
import pandas as pd
import numpy as np

# Set a seed for reproducibility
np.random.seed(42)

# Create course DataFrame
cse_courses = {
    'DBMS': 101,
    'Data Structures': 102,
    'Algorithms': 103,
    'Operating Systems': 104,
    'Computer Networks': 105
}

course = pd.DataFrame({
    'course_id': list(cse_courses.values()),
    'course_name': list(cse_courses.keys())
})

# Assign random courses to students
student['course_id'] = np.random.choice(course['course_id'], size=len(student), replace=True)

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print("Updated Student DataFrame:")
print(student)
print("\nCorrect Course DataFrame:")
print(course)


Updated Student DataFrame:
    student_id        name  marks grade  age  course_id
0            1   Student_1     88     B   21        104
1            2   Student_2     78     B   24        105
2            3   Student_3     64     B   23        103
3            4   Student_4     92     D   23        105
4            5   Student_5     57     D   24        105
5            6   Student_6     70     A   23        102
6            7   Student_7     88     A   20        103
7            8   Student_8     68     D   21        103
8            9   Student_9     72     B   24        103
9           10  Student_10     60     B   21        105
10          11  Student_11     60     A   18        104
11          12  Student_12     73     D   20        103
12          13  Student_13     85     A   22        105
13          14  Student_14     89     A   20        102
14          15  Student_15     73     C   24        104
15          16  Student_16     52     C   22        102
16          17  Stude

In [45]:
# Updated schema with the foreign key on the student table
db_schema_v6 = """
CREATE TABLE student (
    student_id INTEGER,
    name VARCHAR,
    marks INTEGER,
    grade VARCHAR,
    age INTEGER,
    course_id INTEGER
);

CREATE TABLE course (
    course_id INTEGER,
    course_name VARCHAR
);
"""
from transformers import pipeline
model_name = "gaussalgo/T5-LM-Large-text2sql-spider"
text_to_sql_pipeline = pipeline("text2text-generation", model=model_name)
from pandasql import sqldf
def psql(q):
    return sqldf(q, globals())

Device set to use cpu


In [46]:

# V6: Multi-Table Query
nl_query_v6 = "Show student names , their course names."
prompt_v6 = f"### {db_schema_v6}\n### Task\n{nl_query_v6}"
sql_v6 = text_to_sql_pipeline(prompt_v6)

print(f"V6 Natural Language: {nl_query_v6}")
print(f"V6 Generated SQL: {sql_v6[0]['generated_text']}")

# Execute the generated SQL
result_v6 = psql(sql_v6[0]['generated_text'])
print("V6 Query Result on Dummy Data:")
print(result_v6)

V6 Natural Language: Show student names , their course names.
V6 Generated SQL: SELECT T1.name , T2.course_name FROM student AS T1 JOIN course AS T2 ON T1.course_id = T2.course_id
V6 Query Result on Dummy Data:
          name        course_name
0    Student_1  Operating Systems
1    Student_2  Computer Networks
2    Student_3         Algorithms
3    Student_4  Computer Networks
4    Student_5  Computer Networks
5    Student_6    Data Structures
6    Student_7         Algorithms
7    Student_8         Algorithms
8    Student_9         Algorithms
9   Student_10  Computer Networks
10  Student_11  Operating Systems
11  Student_12         Algorithms
12  Student_13  Computer Networks
13  Student_14    Data Structures
14  Student_15  Operating Systems
15  Student_16    Data Structures
16  Student_17  Operating Systems
17  Student_18  Computer Networks
18  Student_19               DBMS
19  Student_20  Operating Systems


In [47]:
# V7: Synonym Handling
nl_query_v7 = "List the names of students with marks above 80."
prompt_v7 = f"### {db_schema_v6}\n### Task\n{nl_query_v7}"
sql_v7 = text_to_sql_pipeline(prompt_v7)

print(f"V7 Natural Language: {nl_query_v7}")
print(f"V7 Generated SQL: {sql_v7[0]['generated_text']}")

# Execute the generated SQL
result_v7 = psql(sql_v7[0]['generated_text'])
print("V7 Query Result on Dummy Data:")
print(result_v7)

V7 Natural Language: List the names of students with marks above 80.
V7 Generated SQL: SELECT name FROM student WHERE marks > 80
V7 Query Result on Dummy Data:
         name
0   Student_1
1   Student_4
2   Student_7
3  Student_13
4  Student_14
5  Student_20


In [48]:
# First, find the marks of Student_5 using pandas
student_5_marks = student[student['name'] == 'Student_5']['marks'].iloc[0]

print(f"Student_5's marks are: {student_5_marks}")

# Use the specific number in the natural language prompt
nl_query_v8 = f"Show the names and marks of students with marks greater than {student_5_marks}."
prompt_v8 = f"### {db_schema_v6}\n### Task\n{nl_query_v8}"
sql_v8 = text_to_sql_pipeline(prompt_v8)

print(f"V8 Natural Language: {nl_query_v8}")
print(f"V8 Generated SQL: {sql_v8[0]['generated_text']}")

# Execute the generated SQL
result_v8 = psql(sql_v8[0]['generated_text'])
print("V8 Query Result on Dummy Data:")
print(result_v8)


Student_5's marks are: 57
V8 Natural Language: Show the names and marks of students with marks greater than 57.
V8 Generated SQL: SELECT name FROM student WHERE marks > 57
V8 Query Result on Dummy Data:
          name
0    Student_1
1    Student_2
2    Student_3
3    Student_4
4    Student_6
5    Student_7
6    Student_8
7    Student_9
8   Student_10
9   Student_11
10  Student_12
11  Student_13
12  Student_14
13  Student_15
14  Student_17
15  Student_19
16  Student_20
